In [4]:
from openai import OpenAI

# Setup
api_key = ""
base_url = "https://gpt.uni-muenster.de/v1"
model = "Llama-3.3-70B"  # model used
client = OpenAI(api_key=api_key, base_url=base_url)


#client.models.list()

In [8]:
# Define scenario
scenario = """
A customer places an order on an e-commerce website. 
The system checks inventory. 
If the item is in stock, customer is informed to pay for the product.
Customer pays for the order.
The payment gateway confirms the payment transaction. 
Finally, the system sends order confirmation email to the customer.
"""

# Create the prompt
prompt = f"""
You are an expert in Subject-Oriented Business Process Modeling using the Parallel Activity Specification Schema (PASS).

Given the following scenario:

\"\"\"{scenario}\"\"\"

1. Identify all the subjects involved.
2. Describe their interactions clearly.
3. Generate the PASS model in two parts:
   - Subject Interaction Diagram (SID): a sequence of message exchanges between subjects
   - Subject Behavior Diagram (SBD): the internal state sequence for each subject

### IMPORTANT:
For each SBD, clearly label the types of states:
- StartState: where the behavior begins
- SendState: when the subject sends a message to another subject
- ReceiveState: when the subject receives a message from another subject
- DoState: internal action or decision (can also act as Start or End if applicable)
- EndState: the subject's behavior concludes
Use the following structured format for the response:
### Subjects:
- Subject A
- Subject B

### Interactions:
1. Subject A -> Subject B: message
2. ...

### SID:
1. Subject A -> Subject B: message
2. ...

### SBD:
#### Subject A:
StartState: Do something
 Description: do the action
SendState: Send message to Subject B
 To:  Subject B
 Msg: message
ReceiveState: Receive message from Subject C
 From: Subject C
 Msg:  message 
DoState: Internal action
 Description: do the action
EndState: Done

#### Subject B:
...

### PASS BEHAVIOR RULES:
For each **SBD**, strictly use the following state types:
- Always give each state a **label**
- **StartState**: where the behavior begins (can be Do, Receive, or Send state).
- **SendState**: when the subject sends/transfers. MUST include `To` and `Msg`.
- **ReceiveState**: when the subject receives a message. MUST include `From` and `Msg`.
- **DoState**: internal action, decision, or processing.
- **EndState**: when the behavior ends.

For 'StartState', do **not** use the type name alone (e.g., "StartState: ReceiveState"). Instead, write:

StartState: <Label>
  Msg: ...
  From/To: ...
  Description: ...

### DEFINITIONS:
- `Msg` must always be a **noun** representing the content of the message (e.g., *order details*, *payment confirmation*).
- `SendState` is used for *sending* or *transferring* a message.
- `ReceiveState` is used for *receiving* a message.
- `DoState` is for *internal actions or decisions* not involving message transfer.
- The **StartState** may be any of: `ReceiveState`, `SendState`, or `DoState`.

IMPORTANT:
- If the `StartState` is a **DoState**, you must provide a `Description`.
- If the `StartState` is a **SendState** or **ReceiveState**, the `Msg` and `To`/`From` fields are sufficient — no separate `Description` is needed.

- **DoState** and Description: used for internal, non-communicative actions of the subject. This is **not** for describing decisions or actions made by other subjects.

  - The **label** should be a concise action verb or short phrase (e.g., 'Decide').
  - The **description** must match what this subject is doing internally — not what another subject (e.g., the system) is doing.

Avoid descriptions that narrate what another subject is doing.
"""

# Make the API call
completion = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=model,
    temperature=0.5
)

# Output
print(completion.choices[0].message.content)

### Subjects:
- Customer
- System
- Payment Gateway

### Interactions:
1. Customer -> System: order details
2. System -> Customer: inventory status
3. Customer -> Payment Gateway: payment information
4. Payment Gateway -> System: payment confirmation
5. System -> Customer: order confirmation

### SID:
1. Customer -> System: order details
2. System -> Customer: inventory status
3. Customer -> Payment Gateway: payment information
4. Payment Gateway -> System: payment confirmation
5. System -> Customer: order confirmation

### SBD:
#### Customer:
StartState: Place Order
  Description: customer places an order on the e-commerce website
SendState: Send Order Details
  To: System
  Msg: order details
ReceiveState: Receive Inventory Status
  From: System
  Msg: inventory status
DoState: Decide on Payment
  Description: customer decides to pay for the order
SendState: Send Payment Information
  To: Payment Gateway
  Msg: payment information
ReceiveState: Receive Order Confirmation
  From: Syst

In [2]:
from openai import OpenAI

# saved response from LLM in variable below named llama_output, 
# otherwise comment it out and do llama_output = completion.choices[0].message.content

llama_output = """
### Subjects:
- Customer
- System
- Payment Gateway

### Interactions:
1. Customer -> System: order details
2. System -> Customer: inventory status
3. Customer -> Payment Gateway: payment information
4. Payment Gateway -> System: payment confirmation
5. System -> Customer: order confirmation

### SID:
1. Customer -> System: order details
2. System -> Customer: inventory status
3. Customer -> Payment Gateway: payment information
4. Payment Gateway -> System: payment confirmation
5. System -> Customer: order confirmation

### SBD:
#### Customer:
StartState: Place Order
  Description: customer places an order on the e-commerce website
SendState: Send Order Details
  To: System
  Msg: order details
ReceiveState: Receive Inventory Status
  From: System
  Msg: inventory status
DoState: Decide on Payment
  Description: customer decides to pay for the order
SendState: Send Payment Information
  To: Payment Gateway
  Msg: payment information
ReceiveState: Receive Order Confirmation
  From: System
  Msg: order confirmation
EndState: Order Confirmed

#### System:
StartState: Receive Order
  From: Customer
  Msg: order details
DoState: Check Inventory
  Description: system checks the inventory
SendState: Send Inventory Status
  To: Customer
  Msg: inventory status
ReceiveState: Receive Payment Confirmation
  From: Payment Gateway
  Msg: payment confirmation
DoState: Process Order
  Description: system processes the order
SendState: Send Order Confirmation
  To: Customer
  Msg: order confirmation
EndState: Order Processed

#### Payment Gateway:
StartState: Receive Payment Information
  From: Customer
  Msg: payment information
DoState: Process Payment
  Description: payment gateway processes the payment
SendState: Send Payment Confirmation
  To: System
  Msg: payment confirmation
EndState: Payment Processed

### PASS BEHAVIOR RULES:
The above SBDs follow the PASS behavior rules, with each state labeled and described according to the rules. The `StartState` is clearly defined for each subject, and the `SendState`, `ReceiveState`, `DoState`, and `EndState` are used as specified. The `Msg` field is used to represent the content of the messages, and the `To` and `From` fields are used to specify the sender and receiver of each message.
"""

In [3]:
import re

def parse_sid(sid_text):
    pattern = re.compile(r"\d+\.\s*(.+?)\s*->\s*(.+?):\s*(.+)")
    sid = []
    for line in sid_text.splitlines():
        print(f"Parsing line: {line}")
       
        match = pattern.search(line)
        if match:
            sender = match.group(1).strip()
            receiver = match.group(2).strip()
            message = match.group(3).strip()
            sid.append((f"{sender} -> {receiver}", message))
        else:
            print("No match found.")
    return sid

def extract_sbd_section(full_text):
    lines = full_text.splitlines()
    sbd_start = None
    for i, line in enumerate(lines):
        if "### SBD" in line:
            sbd_start = i + 1
            break
    if sbd_start is None:
        print("No SBD section found!")
        return ""

    sbd_lines = []
    for line in lines[sbd_start:]:
        if line.strip().startswith("### Explanation"):
            break
        sbd_lines.append(line)

    return "\n".join(sbd_lines).strip()



def parse_sbd(sbd_text):
    print("=== SBD Text to parse ===")
    print(sbd_text)
    print("=========================")
    lines = sbd_text.splitlines()

    sbd = {}
    current_actor = None
    states = {}
    current_state = None
    current_state_num = 0

    actor_header_re = re.compile(r"^\s*####\s*(.+):$")
  
    state_header_re = re.compile(r"^\s*(?:\d+\.\s*|[-*]\s*)?(\w+State):\s*(.*)")
    
    for idx, line in enumerate(lines):
        line = line.rstrip()
        if not line:
            continue

        # Detect actor header like: #### Customer:
        actor_match = actor_header_re.match(line)
        if actor_match:
            if current_actor:
                sbd[current_actor] = list(states.values())
            current_actor = actor_match.group(1).strip()
            states = {}
            current_state = None
            current_state_num = 0
            continue

        if current_actor is None:
            continue  # Skip anything before the first actor header

        # Detect numbered state line like: 1. StartState: 
        state_match = state_header_re.match(line)
        if state_match:
            state_type = state_match.group(1)
            description = state_match.group(2).strip()
            current_state_num += 1
            
            current_state = {
                "num": current_state_num,
                "type": state_type,
                "description": description
            }
            states[current_state_num] = current_state
            continue

        if current_state is None:
            continue

        line_stripped = line.strip()

        # Capture From/To/Msg in SendState, ReceiveState, and StartState
        if current_state["type"] in ["SendState", "StartState"]:
            if line_stripped.startswith("To:"):
                current_state["To"] = line.split(":", 1)[1].strip()
            elif line_stripped.startswith("Msg:"):
                current_state["Msg"] = line.split(":", 1)[1].strip()

        if current_state["type"] in ["ReceiveState", "StartState"]:
            if line_stripped.startswith("From:"):
                current_state["From"] = line.split(":", 1)[1].strip()
            elif line_stripped.startswith("Msg:"):
                current_state["Msg"] = line.split(":", 1)[1].strip()

        # Capture Description or Action for DoState and StartState
        if current_state["type"] in ["DoState", "StartState"]:
            if line_stripped.startswith("Description:"):
                current_state["Description"] = line.split(":", 1)[1].strip()

    if current_actor:
        sbd[current_actor] = list(states.values())

    return sbd


# 1. Extract sections
sbd_text = extract_sbd_section(llama_output)

# 2. Parse
parsed_sbd = parse_sbd(sbd_text)

sid_section_start = llama_output.find("### SID:")
sid_section_end = llama_output.find("### SBD:")
sid_text = llama_output[sid_section_start:sid_section_end].replace("### SID:", "").strip()
parsed_sid = parse_sid(sid_text)

# 3. Assemble full process
process = {
    "SID": parsed_sid,
    "SBD": parsed_sbd
}

print("Parsed SBD::", parsed_sbd)
print("Parsed SID::", parsed_sid) 

=== SBD Text to parse ===
#### Customer:
StartState: Place Order
  Description: customer places an order on the e-commerce website
SendState: Send Order Details
  To: System
  Msg: order details
ReceiveState: Receive Inventory Status
  From: System
  Msg: inventory status
DoState: Decide on Payment
  Description: customer decides to pay for the order
SendState: Send Payment Information
  To: Payment Gateway
  Msg: payment information
ReceiveState: Receive Order Confirmation
  From: System
  Msg: order confirmation
EndState: Order Confirmed

#### System:
StartState: Receive Order
  From: Customer
  Msg: order details
DoState: Check Inventory
  Description: system checks the inventory
SendState: Send Inventory Status
  To: Customer
  Msg: inventory status
ReceiveState: Receive Payment Confirmation
  From: Payment Gateway
  Msg: payment confirmation
DoState: Process Order
  Description: system processes the order
SendState: Send Order Confirmation
  To: Customer
  Msg: order confirmation


In [4]:
###########################SBD Graph##########################

import os  
os.environ["PATH"] += os.pathsep + "/home/s/smanan/.conda/envs/myenv/bin"

from graphviz import Digraph

state_styles = {
    'StartState': {"shape": "rectangle", "color": "yellow"},
    'EndState':   {"shape": "rectangle", "color": "yellow"},
    'SendState':  {"shape": "rectangle", "color": "green"},
    'ReceiveState': {"shape": "rectangle", "color": "pink"},
    'DoState':    {"shape": "rectangle", "color": "yellow"},
    'Unknown':    {"shape": "rectangle", "color": "yellow"},
}

for subject, steps in parsed_sbd.items():
    dot = Digraph(name=subject, format='pdf')
    dot.attr(rankdir='LR')

    step_map = {step["num"]: step for step in steps}
    step_nums = sorted(step_map.keys())

    # Draw nodes
    for step in steps:
        label = step["description"]

        # Determine if StartState is acting as ReceiveState
        if step["type"] == "StartState" and "From" in step and "Msg" in step and "receive" in step["description"].lower():
            style = state_styles["ReceiveState"]
        else:
            style = state_styles.get(step["type"], state_styles["Unknown"])

        dot.node(str(step["num"]), label=label, shape=style["shape"], style="filled", fillcolor=style["color"])

    # Draw transitions
    for i, step in enumerate(steps):
        current_num = str(step["num"])

        if step["type"] == "EndState":
            continue

        label = ""
        next_step_num = step_nums[i + 1] if i + 1 < len(step_nums) else None

        if step["type"] == "SendState":
            label = f"To={step.get('To', '')}\\nMsg={step.get('Msg', '')}"
        elif step["type"] == "ReceiveState":
            label = f"From={step.get('From', '')}\\nMsg={step.get('Msg', '')}"
        elif step["type"] == "DoState":
            label = step.get("Description", step["Description"])
        elif step["type"] == "StartState":
            # Support From/To/Msg in StartState if present
            from_part = f"From={step.get('From', '')}" if "From" in step else ""
            to_part = f"To={step.get('To', '')}" if "To" in step else ""
            msg_part = f"Msg={step.get('Msg', '')}" if "Msg" in step else ""

            parts = [from_part, to_part, msg_part]
            parts = [p for p in parts if p]
            
            label = "\\n".join(parts) if parts else step.get("Description", step.get("Action", step["description"]))

        if next_step_num:
            dot.edge(current_num, str(next_step_num), label=label)

    safe_subject = subject.replace(" ", "_")
    filename = f"/home/s/smanan/sbd_llama_{safe_subject}"
    dot.render(filename, cleanup=True)


In [5]:
import os #########################SID Graph###################
os.environ["PATH"] += os.pathsep + "/home/s/smanan/.conda/envs/myenv/bin"

from graphviz import Digraph

def draw_sid_graph(sid_list, output_path="sid_graph"):
    dot = Digraph(comment="SID - Sequence Interaction Diagram")
    dot.attr(rankdir='LR',splines='polyline')  # Left to right direction

    for interaction, action in sid_list:
        source, target = [s.strip() for s in interaction.split("->")]
        dot.node(source, source, shape="box", style="filled", fillcolor="lightblue")
        dot.node(target, target, shape="box", style="filled", fillcolor="lightgreen")
        dot.edge(source, target, label=action)

    # Render graph to file
    dot.render(output_path, format='png', cleanup=True)
    print(f"SID graph saved to: {output_path}.png")

In [6]:
draw_sid_graph(process["SID"], output_path="/home/s/smanan/SID_Visualization_Llama")

SID graph saved to: /home/s/smanan/SID_Visualization_Llama-July12.png
